In [1]:
import numpy as np
from scipy.spatial.transform import Rotation 

In [5]:
joy_y = [0,1,0]
correction_x = [1,-1,0]

alignment = np.dot(correction_x, joy_y)
sign = np.sign(alignment)
sign

-1

In [2]:
R_b = Rotation.from_quat([0, 1, 1, 1]).as_matrix()
R_b

array([[-0.33333333, -0.66666667,  0.66666667],
       [ 0.66666667,  0.33333333,  0.66666667],
       [-0.66666667,  0.66666667,  0.33333333]])

In [3]:
angle = 1.57
R_ab = np.array([[np.cos(angle), -np.sin(angle), 0],
                         [np.sin(angle), np.cos(angle) , 0],
                         [0            , 0             , 1]])

In [4]:
R_a = np.dot(R_ab, R_b)
q_a = Rotation.from_matrix(R_a).as_quat()
q_a

array([-4.08085709e-01,  4.08410808e-01,  8.16496516e-01,  3.25099044e-04])

In [5]:
def quaternion_to_euler(quaternion):
        
        # Create a Rotation object from the quaternion
        r = Rotation.from_quat(quaternion)
        
        # Convert Rotation object to Euler angles (roll, pitch, yaw)
        euler_angles = r.as_euler('xyz')  # Specify 'xyz' order for yaw, pitch, roll
        
        return euler_angles


In [6]:
quaternion = [0.7071, 0.0, 0.0, 0.7071]  # Example quaternion [w, x, y, z]
euler_angles = quaternion_to_euler(quaternion)
print("Euler angles (roll, pitch, yaw) in radians:", euler_angles)

Euler angles (roll, pitch, yaw) in radians: [1.57079633 0.         0.        ]


In [7]:
import os

data_dir = "/home/shalutha/TLGC_data/iros_demo1"
model_dir = data_dir +f"/model"

items = os.listdir(model_dir)
        

filename = model_dir + f"/{items[0]}"
model = np.load(filename)

print(model["vertical_start"])

False


In [8]:
for i in range (4,0,-1):
    print (i)

4
3
2
1


In [9]:
for x in range(6):
  if x == 7: 
    break
  print(x)
else:
  print("Finally finished!")

0
1
2
3
4
5
Finally finished!


In [10]:
a = np.sqrt(4**2 + 3**2)
a

5.0

In [11]:
a = []

for i in a:
    print("a")

In [12]:
a = [1,2,3,4]
b = np.vstack(a)
a, b

([1, 2, 3, 4],
 array([[1],
        [2],
        [3],
        [4]]))

In [13]:
a = [1,23,3]
b = np.array(a)
b

array([ 1, 23,  3])

In [18]:
import numpy as np

# Function to compute the 2x2 transformation matrix at point t
def compute_transformation_matrix(a, b):
    # We only consider the X and Y components for the joystick input mapping
    a_xy = a
    b_xy = b
    
    # Joystick frame unit vectors in 2D
    i = np.array([0,1,0])
    j = np.array([-1,0,0])

    
    # Construct the 2x2 transformation matrix T using dot products
    T = np.array([
        [np.dot(a_xy, i), np.dot(a_xy, j)],
        [np.dot(b_xy, i), np.dot(b_xy, j)]
    ])
    
    return T

# Example correction axes in 3D
# a = np.array([0.707, 0.707, 0])  # Example axis at 45 degrees in the X-Y plane
# b = np.array([0, 0, 1])          # Vertical axis along Z
a = np.array([ 0.99922177, -0.01938759,  0.03435075])
b = np.array([-7.02332609e-06, 8.70779838e-01, 4.91673137e-01])

# Compute the transformation matrix
T = compute_transformation_matrix(a, b)
print("Transformation Matrix:\n", T)

# Function to map joystick inputs to correction axes movements
def map_joystick_to_correction_axes(T, joystick_input):
    u, v = joystick_input
    # Compute the movements along correction axes
    movement = np.dot(T, np.array([u, v]))
    return movement

# Example usage with 2D joystick input
joystick_input = [0.0, -0.0001833726465702057]  # User pushes joystick to the right

movement = map_joystick_to_correction_axes(T, joystick_input)
print("Movement along correction axes:", movement)


Transformation Matrix:
 [[-1.93875900e-02 -9.99221770e-01]
 [ 8.70779838e-01  7.02332609e-06]]
Movement along correction axes: [ 1.83229940e-04 -1.28788589e-09]


In [ ]:
x and y after mult  [-0.  0. -0.] [ 1.28788589e-09 -1.59677203e-04 -9.01594045e-05]

In [31]:
import numpy as np

# Function to compute the dot product projections of joystick axes onto correction axes
def compute_axis_projections(correction_axes, joy_x, joy_y):
    # Project joystick X and Y axes onto correction axes
    projections_x = [np.dot(joy_x, axis) for axis in correction_axes]
    projections_y = [np.dot(joy_y, axis) for axis in correction_axes]
    return projections_x, projections_y

# Function to select the most aligned correction axis based on projections
def select_aligned_axis(projections):
    # Find the axis with the maximum absolute projection
    max_index = np.argmax(np.abs(projections))
    direction = np.sign(projections[max_index])
    return max_index, direction

# Example correction axes in 3D
correction_axes = [
    np.array([1, -2, 0]),  # Example axis at 45 degrees in the X-Y plane
    np.array([-2, -1, 0])           # Vertical axis along Z
]

# Define the fixed joystick axes in the global frame
joy_x = np.array([0, 1, 0])  # Joystick X-axis (global frame)
joy_y = np.array([-1, 0, 0]) # Joystick Y-axis (global frame)

# Compute the projections of the joystick axes onto the correction axes
projections_x, projections_y = compute_axis_projections(correction_axes, joy_x, joy_y)
# print("Projections on X-axis:", projections_x)
# print("Projections on Y-axis:", projections_y)

# Select the most aligned correction axis for joystick X-axis
aligned_axis_index_x, direction_x = select_aligned_axis(projections_x)
# print("Aligned Axis Index for Joystick X-axis:", aligned_axis_index_x)
# print("Direction for Joystick X-axis:", direction_x)

# Select the most aligned correction axis for joystick Y-axis
aligned_axis_index_y, direction_y = select_aligned_axis(projections_y)
# print("Aligned Axis Index for Joystick Y-axis:", aligned_axis_index_y)
# print("Direction for Joystick Y-axis:", direction_y)

# The most aligned correction axes
aligned_axis_x = correction_axes[aligned_axis_index_x]
aligned_axis_y = correction_axes[aligned_axis_index_y]
print("Aligned Axis for Joystick X-axis:", aligned_axis_x)
print("Aligned Axis for Joystick Y-axis:", aligned_axis_y)

# Example movement magnitudes based on joystick input
joystick_input = np.array([0, -1])  # Example diagonal movement

# Determine the 3D movement based on the most aligned correction axes
movement_x = direction_x * joystick_input[0] * aligned_axis_x
movement_y = direction_y * joystick_input[1] * aligned_axis_y
total_movement = movement_x + movement_y
print("Total Movement in 3D space:", total_movement)


Aligned Axis for Joystick X-axis: [ 1 -2  0]
Aligned Axis for Joystick Y-axis: [-2 -1  0]
Total Movement in 3D space: [2 1 0]


In [ ]:
correction_axes = [
    np.array([1, 0, 0]),  # Example axis at 45 degrees in the X-Y plane
    np.array([0, -1, 0])           # Vertical axis along Z
]

In [ ]:
### check the above code with examples

## remember if the both axes becomes one then we have to ensure that for the other axis we will select the othe one

In [36]:
    
def project_onto_plane(v, normal):
    # Normalize the normal vector to ensure it's a unit vector
    normal_normalized = normal / np.linalg.norm(normal)
    # Calculate the projection of v onto the plane defined by the normal
    projection = v - np.dot(v, normal_normalized) * normal_normalized
    return projection

In [34]:
import numpy as np

# Function to compute the dot product projections of joystick axes onto correction axes
def compute_axis_projections(correction_axes, joy_x, joy_y):
    # Project joystick X and Y axes onto correction axes
    projections_x = [np.dot(joy_x, axis) for axis in correction_axes]
    projections_y = [np.dot(joy_y, axis) for axis in correction_axes]
    return projections_x, projections_y

# Function to select the most aligned correction axis based on projections
def select_aligned_axis(projections):
    # Find the axis with the maximum absolute projection
    max_index = np.argmax(np.abs(projections))
    direction = np.sign(projections[max_index])
    return max_index, direction

# Example correction axes in 3D
# Example correction axes in 3D
# correction_axes = [
#     np.array([1, -2, 0]),  # Example axis at 45 degrees in the X-Y plane
#     np.array([-2, -1, 0])           # Vertical axis along Z
# ]

correction_axes = [
    np.array([1, 0, 0]),  # Example axis at 45 degrees in the X-Y plane
    np.array([0, -1, 0])           # Vertical axis along Z
]




# Define the fixed joystick axes in the global frame
joy_x = np.array([0, 1, 0])  # Joystick X-axis (global frame)
joy_y = np.array([-1, 0, 0]) # Joystick Y-axis (global frame)

# Compute the projections of the joystick axes onto the correction axes
projections_x, projections_y = compute_axis_projections(correction_axes, joy_x, joy_y)

# Select the most aligned correction axis for joystick X-axis
aligned_axis_index_x, direction_x = select_aligned_axis(projections_x)


# Select the most aligned correction axis for joystick Y-axis
aligned_axis_index_y, direction_y = select_aligned_axis(projections_y)
print("Aligned Axis Index for Joystick Y-axis:", aligned_axis_index_y)
print("Direction for Joystick Y-axis:", direction_y)



# The most aligned correction axes, invert if direction is negative
aligned_axis_x = correction_axes[aligned_axis_index_x] * direction_x
aligned_axis_y = correction_axes[aligned_axis_index_y] * direction_y
print("Aligned Axis for Joystick X-axis:", aligned_axis_x)
print("Aligned Axis for Joystick Y-axis:", aligned_axis_y)

# Example movement magnitudes based on joystick input
joystick_input = np.array([0.006, -0.006])  # Example diagonal movement

# # Determine the 3D movement based on the most aligned correction axes
# movement_x = joystick_input[0] * aligned_axis_x
# movement_y = joystick_input[1] * aligned_axis_y
# total_movement = movement_x + movement_y
# print("Total Movement in 3D space:", total_movement)


Aligned Axis Index for Joystick Y-axis: 0
Direction for Joystick Y-axis: -1
Aligned Axis for Joystick X-axis: [0 1 0]
Aligned Axis for Joystick Y-axis: [-1  0  0]
Total Movement in 3D space: [0.006 0.006 0.   ]


In [ ]:
def vertical_check(self, eT, model):
    
    global_z = np.array([0, 0, 1])

    z_diff = self.calculate_angle_between_vectors(eT, global_z)


    if z_diff < 90:
        if z_diff > 45:
            return True
        else:
            return False
    else:
        if z_diff < 145:
            return True
        else:
            return False

In [58]:
import numpy as np

# Function to compute the dot product projections of joystick axes onto correction axes
def compute_axis_projections(correction_axes, joy_x, joy_y):
    # Project joystick X and Y axes onto correction axes
    projections_x = [np.dot(joy_x, axis) for axis in correction_axes]
    projections_y = [np.dot(joy_y, axis) for axis in correction_axes]
    return projections_x, projections_y

# Function to select the most aligned correction axis based on projections
def select_aligned_axis(projections):
    # Find the axis with the maximum absolute projection
    max_index = np.argmax(np.abs(projections))
    direction = np.sign(projections[max_index])
    return max_index, direction

# Example correction axes in 3D
# Example correction axes in 3D
# correction_axes = [
#     np.array([1, -2, 0]),  # Example axis at 45 degrees in the X-Y plane
#     np.array([-2, -1, 0])           # Vertical axis along Z
# ]

# Define the fixed joystick axes in the global frame ASSUMIN WE ARE IN FRONT

joy_x = np.array([0, 1, 0])  # Joystick X-axis (global frame)
joy_y = np.array([-1, 0, 0]) # Joystick Y-axis (global frame)


correction_x = [0,-1,0]
correction_y = [0,0,1]

def map_input(correction_x, correction_y, joy_x, joy_y):

    correction_axes = [
        correction_x,  
        correction_y            
    ]

    z  = np.array([0,0,1])

    projected_corr_x = project_onto_plane(correction_axes[0], z ) 
    projected_corr_y = project_onto_plane(correction_axes[1], z ) 


    projected_corr_axes = np.array([projected_corr_x, projected_corr_y])


    # Compute the projections of the joystick axes onto the correction axes
    projections_x, projections_y = compute_axis_projections(projected_corr_axes, joy_x, joy_y)

    # Select the most aligned correction axis for joystick X-axis
    aligned_axis_index_x, direction_x = select_aligned_axis(projections_x)


    # Select the most aligned correction axis for joystick Y-axis
    aligned_axis_index_y, direction_y = select_aligned_axis(projections_y)

# print("Aligned Axis Index for Joystick Y-axis:", aligned_axis_index_y)
# print("Direction for Joystick Y-axis:", direction_y)



    # The most aligned correction axes, invert if direction is negative
    aligned_axis_x = correction_axes[aligned_axis_index_x] 
    aligned_axis_y = correction_axes[aligned_axis_index_y]

    if (aligned_axis_x == projected_corr_x).all():
        aligned_axis_x = correction_axes[0]
        aligned_axis_y = correction_axes[1]
    else:
        aligned_axis_x = correction_axes[1]
        aligned_axis_y = correction_axes[0]


    ### HANDLING FOR WHEN A CORRECTION AXIS IS FULLY ALIGNED WITH Z ####
    if (projected_corr_x ==np.array([0,0,0])).all():
        print("#### corr x got aligned with z")
        aligned_axis_y = np.array([0,0,1])
        direction_y = 1.0

    if (projected_corr_y ==np.array([0,0,0])).all():
        print("#### corr y got aligned with z")
        aligned_axis_y = np.array([0,0,1])
        direction_y = 1.0


    print("original correction axes", correction_axes[0], correction_axes[1])
    print("projected correction axes", projected_corr_x, projected_corr_y)

    print("Aligned Axis for Joystick X-axis:", aligned_axis_x, " Direction identifier", direction_x)
    print("Aligned Axis for Joystick Y-axis:", aligned_axis_y,  " Direction identifier", direction_y)

    return aligned_axis_x, aligned_axis_y, direction_x, direction_y


aligned_axis_x, aligned_axis_y, direction_x, direction_y = map_input(correction_x, correction_y, joy_x, joy_y)

# Example movement magnitudes based on joystick input
joystick_input = np.array([0.006, -0.006])  # Example diagonal movement

# # Determine the 3D movement based on the most aligned correction axes

aligned_axis_x = np.array(aligned_axis_x)
aligned_axis_y = np.array(aligned_axis_y)

movement_x = joystick_input[0] * aligned_axis_x * direction_x
movement_y = joystick_input[1] * aligned_axis_y * direction_y
# total_movement = movement_x + movement_y
# print("Total Movement in 3D space:", total_movement)


#### corr y got aligned with z
original correction axes [0, -1, 0] [0, 0, 1]
projected correction axes [ 0. -1.  0.] [0. 0. 0.]
Aligned Axis for Joystick X-axis: [0, -1, 0]  Direction identifier -1.0
Aligned Axis for Joystick Y-axis: [0 0 1]  Direction identifier 1.0


In [47]:
v = project_onto_plane(np.array([0,0,1]), z ) 
v

array([0., 0., 0.])

In [ ]:
def optimize_orientation(self, i, gc_circles, directrix, eT):

    center2 = directrix[i+1]
    center1 = directrix[i]  

    normal2 = eT[i+1]
    normal1 = eT[i]

    # circle2 = gc_circles[i+1,:,:]
    circle1 = gc_circles[i,:,:]

    intersection_exist = False

    for point in circle1:
        vector_pointing_to_directrix = (point[0] - center2[0], point[1] - center2[1], point[2] - center2[2])
        theta_val = self.calculate_angle_between_vectors(normal2, vector_pointing_to_directrix) #in degrees
        if theta_val < 90: ##Intersection exist
            print("intersection exist in", i)
            intersection_exist = True
            break
            ## check the current angle between the two planes/normal vector and the rotated normal
            # angle_between_two_planes = self.relative_angle_between_planes(normal1, normal2)
            # penalty = angle_between_two_planes
            
            

    if intersection_exist:
        initial_guess = self.relative_angle_between_planes(normal1, normal2)
        # print("initial angle between", i, "and ", i+1, " th cross sections", initial_guess)
        
        initial_guess = initial_guess*0.1 #initially we will adjust 10% from the initial angle between the two planes
        
    
        result = minimize(self.objective_function, initial_guess, args=(normal1, circle1, normal2, center2, i), bounds=[(-np.pi / 4, np.pi / 4)])

        optimized_angle = result.x[0]
        # print("the angle needed to rotate", optimized_angle)
        axis = np.cross(normal1, normal2)
        optimized_normal2 = self.rodrigues_rotation(normal2, axis, optimized_angle)

        angle_after_opt = self.relative_angle_between_planes(normal1, optimized_normal2)
        # print("######### after opt angle between", i, "and ", i+1, " th cross sections", angle_after_opt)

        return optimized_normal2
    
    else:
        print("No intersection returning the previous normal")
        return normal2

In [ ]:
    def check_intersections_simple(self, circle1, center2, normal1, normal2, i):

        for point in circle1:
            
            vector_pointing_to_directrix = (point[0] - center2[0], point[1] - center2[1], point[2] - center2[2])

            theta_val = self.calculate_angle_between_vectors(normal2, vector_pointing_to_directrix) #in degrees

            if theta_val < 90: ##Intersection exist

                # print("intersection exist in", i)

                ## check the current angle between the two planes/normal vector and the rotated normal
                angle_between_two_planes = self.relative_angle_between_planes(normal1, normal2)
                penalty = angle_between_two_planes
                return True, penalty

        else:
            # print("No intersection")
            return False, None

In [ ]:
def optimize_orientation_scipy(self, i, gc_circles, directrix, eT):
    center2 = directrix[i + 1]
    normal2 = eT[i + 1]
    normal1 = eT[i]
    circle1 = gc_circles[i, :, :]

    prev_unit_rotated_normal2 = None

    initial_angle_between_two_planes = self.relative_angle_between_planes(normal1, normal2)

    intersection_exist = self.check_intersections_geometric(circle1, center2, normal2)

    init_in_degrees = np.degrees(initial_angle_between_two_planes)

    if intersection_exist:
        print("Geometric: Intersection exists in", i, "and", i + 1, "with the initial angle", np.degrees(initial_angle_between_two_planes))
        
        def objective(angle):
            axis = np.cross(normal2, normal1)
            if np.linalg.norm(axis) == 0:
                return prev_unit_rotated_normal2
            
            unit_axis = axis / np.linalg.norm(axis)

            # angle = np.radians(init_in_degrees - angle) ##Not sure whether this is needed or nor

            rotated_normal2 = self.rodrigues_rotation(normal2, unit_axis, np.radians(angle))
            unit_rotated_normal2 = rotated_normal2 / np.linalg.norm(rotated_normal2)

            new_angle = self.relative_angle_between_planes(normal1, rotated_normal2) #onl for debugging
            print ("New angle between the two planes", np.degrees(new_angle))

            intersection_exist_after_rotation = self.check_intersections_geometric(circle1, center2, unit_rotated_normal2)
            
            prev_unit_rotated_normal2 = unit_rotated_normal2

            # return 0 if not intersection_exist_after_rotation else np.degrees(self.relative_angle_between_planes(normal1, unit_rotated_normal2))
            if not intersection_exist_after_rotation:
                return 0
            
            else:
                return np.degrees(self.relative_angle_between_planes(normal1, unit_rotated_normal2))

        result = minimize(objective, x0=np.degrees(initial_angle_between_two_planes), bounds=[(0, np.degrees(initial_angle_between_two_planes))])
        
        optimized_angle = result.x[0]
        print("final angle", optimized_angle, "initial angle", init_in_degrees) #check this and see whether the angle has been reduced
        ## based on that decide whether to directly use this angle or reduce it from the initial angle like below
        # optimized_angle = np.radians(init_in_degrees - optimized_angle) ##Not sure whether this is needed or not

        axis = np.cross(normal2, normal1)
        unit_axis = axis / np.linalg.norm(axis)
        optimized_rotated_normal2 = self.rodrigues_rotation(normal2, unit_axis, np.radians(optimized_angle))
        unit_rotated_normal2 = optimized_rotated_normal2 / np.linalg.norm(optimized_rotated_normal2)

        if not self.check_intersections_geometric(circle1, center2, unit_rotated_normal2):
            print(f"Intersection resolved with optimized angle {optimized_angle} degrees")
            return unit_rotated_normal2
        else:
            print("############### couldn't find an angle without intersection at", i, "so returning the closest vector")
            return unit_rotated_normal2

    print("No intersection, returning the previous normal", i)
    return normal2

In [ ]:
repro_trajectories = reproduce(s_model, numRepro, starting, initPoints, Ratio, crossSectionType, strategy, direction)
trajectory_xyz = repro_trajectories[0]

trajectory_q = s_model["q"]
self.clear_rviz_pub.publish("reproduced")
self.visualise_trajectory(trajectory_xyz, trajectory_q)

nb_points = trajectory_xyz.shape[0]

if self.physical_robot : self.myp_app_pub.publish("start")
PcurrG = trajectory_xyz[0,:]
QcurrG = trajectory_q[0,:]
q_weight = q_weights[0,:]

self.pub_cmd_pose(PcurrG, QcurrG, q_weight)

for i in range(nb_points):
    PcurrG = trajectory_xyz[i,:]
    QcurrG = trajectory_q[i,:]
    q_weight = q_weights[i,:]
    
    self.pub_cmd_pose(PcurrG, QcurrG, q_weight)
    AcurrG = PcurrG - model["directrix"][current_idx,:]
    Ratio = np.array([np.sqrt(AcurrG[0]**2 + AcurrG[1]**2 + AcurrG[2]**2) / model["Rc"][current_idx]])
    self.pub_surface_pose(i, current_idx, PcurrG, AcurrG, QcurrG, q_weight, Ratio, "")
    # i designates the NEXT idx of the robot
    if i != 0: current_idx += 1*direction # Designates the idx AT which the robot is located
    self.rate.sleep()

In [ ]:

    def saturate_model_vertical(self, model): #Verticalizing the ends only for the 5% of all the cicles 

        gc_circles = model["GC"]
        nb_points = gc_circles.shape[0]
        self.saturate_start_idxs = []
        self.saturate_end_idxs = []

        global_z = np.array([0, 0, 1])

        eT = model["eT"]
        eN = model["eN"]
        eB = model["eB"]
        directrix = model["directrix"]
        Rc = model["Rc"]

      
        if self.vertical_start:

            tangent_at_start = eT[0,:] / np.linalg.norm(eT[0,:]) #
            projected_vertical_start_tangent = self.project_onto_plane(tangent_at_start, global_z)
            projected_vertical_start_tangent = projected_vertical_start_tangent / np.linalg.norm(projected_vertical_start_tangent)

            eT[0,:] = projected_vertical_start_tangent

            #verticalizing the first circle
            gc_circles, eT,eN, eB = self.make_vertical(-1, gc_circles, Rc, eT, eN, eB, directrix)
            self.saturate_start_idxs.append(0)
            ### Here we have to ensure that the starting circle is orthogonal to the projected vector ::: Need to write a verticalizing function

            directrix_at_start = directrix[0]
            
            for i in range(int(nb_points/20)):
                circle = gc_circles[i+1,:,:]
                for point in circle:
                    vector_pointing_to_directrix = (directrix_at_start[0] - point[0], directrix_at_start[1] - point[1], directrix_at_start[2] - point[2])

                    theta_val = self.calculate_angle_between_vectors(projected_vertical_start_tangent, vector_pointing_to_directrix) #in degrees

                    if theta_val < 90: ##Intersection exist
                        self.saturate_start_idxs.append(i+1)
                        break
                
        if self.vertical_end:

            tangent_at_end = eT[-1,:] / np.linalg.norm(eT[-1,:]) #
            projected_vertical_end_tangent = self.project_onto_plane(tangent_at_end, global_z)
            projected_vertical_end_tangent = projected_vertical_end_tangent / np.linalg.norm(projected_vertical_end_tangent)

            eT[-1,:] = projected_vertical_end_tangent

            #verticalizing the final circle
            gc_circles, eT,eN, eB = self.make_vertical(-1, gc_circles, Rc, eT, eN, eB, directrix)
            self.saturate_end_idxs.append(nb_points-1)


            directrix_at_end = directrix[-1]

            # print("Found a vertical end and the shape of gc_circles is", gc_circles.shape)

            for j in range(nb_points-1, int(19*nb_points/20), -1):
                circle = gc_circles[j-1,:,:]
                for point in circle:
                    vector_pointing_to_directrix = (directrix_at_end[0] - point[0], directrix_at_end[1] - point[1], directrix_at_end[2] - point[2])

                    theta_val = self.calculate_angle_between_vectors(projected_vertical_end_tangent, vector_pointing_to_directrix) #in degrees

                    if theta_val < 90: ##Intersection exist
                        # print("intersection exist")
                        self.saturate_end_idxs.append(j-1)
                        break

        if len(self.saturate_start_idxs) != 0: 
            for i in self.saturate_start_idxs:
                print("saturate_start_idxs", self.saturate_start_idxs)
                gc_circles, eT,eN, eB = self.make_vertical(i, gc_circles, Rc, eT, eN, eB, directrix, location = "start")


        if len(self.saturate_end_idxs) != 0: 
            for i in self.saturate_end_idxs:
                print("saturate_end_idxs", self.saturate_end_idxs)
                gc_circles, eT,eN, eB = self.make_vertical(i, gc_circles, Rc, eT, eN, eB, directrix, location = "end")


        if len(self.saturate_end_idxs) != 0 or len(self.saturate_start_idxs) != 0:
            for i in range(0, nb_points-1):
                optimized_normal2 = self.optimize_orientation_geometric(i, gc_circles, directrix, eT)
                
                gc_circles, eT, eN, eB = self.adjust_TNB(i+1, optimized_normal2, gc_circles, Rc, eT, eN, eB, directrix)


        # Update model
        model["GC"] = gc_circles
        model["eT"] = eT
        model["eN"] = eN
        model["eB"] = eB

        rospy.loginfo(f"  Model saturation completed.")
         
        return model

In [ ]:
    def make_horizontal(self, i, gc_circles, Rc, eT, eN, eB, directrix, global_x, global_y, global_z):

        T = eT[i,:]/np.linalg.norm(eT[i,:])
        cos_angle_z = np.clip(np.dot(T, global_z), -1.0, 1.0)
        angle_z = np.arccos(cos_angle_z)
        angle_z = np.degrees(angle_z)

        if angle_z > 90: 
            T = -global_z
        else: 
            T = global_z
        
        # Redefine GC
        gc_circles[i,:,:] = circular_GC(np.array([directrix[i,:]]), np.array([global_x]), 
                                        np.array([global_y]), np.array([Rc[i]]))
        
        # Change N and B
        N = self.project_onto_plane(eN[i,:], global_z)


        # Normalise
        T = T/np.linalg.norm(T)
        N = N/np.linalg.norm(N)
        B = np.cross(N, T)
        B = B/np.linalg.norm(B)

        # Assign
        eT[i,:] = T
        eN[i,:] = N
        eB[i,:] = B

        return gc_circles, eT, eN, eB